In [1]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA
prob = om.Problem()
prob.model = SellarMDA()
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-8)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()

In [2]:
from ipytree import Tree, Node
from ipywidgets import Button, Label, FloatText, HBox, VBox, Text, Output, Layout
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
from openmdao.core.component import Component
def system_iter(sys, node):
    if sys.name == '_auto_ivc':
        return
    name = sys.name if sys.name else 'root'
    new_node = Node(name)
    node.add_node(new_node)

    if isinstance(sys, Component):
        inputs = list(sys._var_allprocs_prom2abs_list['input'].keys())
        for input in inputs:
            input_node = Node(input)
            new_node.add_node(input_node)
    else:
        for s in sys._subsystems_myproc:
            system_iter(s, new_node)


In [ ]:
label1 = widgets.Label('label')
output2 = widgets.Output()

display(label1, output2)

def on_value_change(change):
    with output2:
        print(change['new'])

int_range.observe(on_value_change, names='value')

In [4]:
def update_prob_val(change):
    with output2:
        print(change['new'], change['owner'].description)
    prob[change['owner'].description] = change['new']

In [5]:
output2 = widgets.Output()
display(output2)
def on_selected_change(change):
    with output2:
        print(change['new'])
        val_widget = widgets.FloatText(
            value=7.5,
            description=change['new'][0].name,
            disabled=False
        )
        val_widget.observe(update_prob_val, 'value')
        value_widget_box.children += (val_widget,)

Output()

In [14]:
output = Output()

@output.capture()
def run_handler(change):
    output.clear_output()
    print('Running driver')
    prob.run_driver()
    prob.list_problem_vars()


run_driver_button = Button(description='Run driver')
run_driver_button.on_click(run_handler)

In [15]:

value_widget_box = VBox([Label("Model Variables"),run_driver_button])

In [16]:
def select_vars_to_set_old(prob):
    tree = Tree(stripes=True, multiple_selection=True)
    system_iter(prob.model, tree)
    tree.observe(on_selected_change, names='selected_nodes')
    widget = HBox([tree, value_widget_box,output])
    return widget

In [17]:
select_vars_to_set_old(prob)

In [22]:
def set_model_vars(prob, value_widget_box):
    for var_widget in value_widget_box.children[3:]:
        prob[var_widget.description] = var_widget.value

In [ ]:
set_model_vars(prob,value_widget_box)

In [ ]:
prob['y2']

In [18]:
value_widget_box = VBox([Label("Model Variables"),])
def select_vars_to_set(prob):
    tree = Tree(stripes=True, multiple_selection=True)
    system_iter(prob.model, tree)
    tree.observe(on_selected_change, names='selected_nodes')
    widget = HBox([tree, value_widget_box])
    return widget

In [20]:
select_vars_to_set(prob)

In [27]:
set_model_vars(prob,value_widget_box)

In [29]:
prob['y2']

array([200.])